################################################################################
#
# 動的予測モデルとベースラインモデルの性能比較分析
#
# --- 概要 ---
# 本スクリプトは、個人の経時的データを用いて翌日のイベント発生を予測するため、
# 4種類の機械学習モデルを構築し、その予測性能を5分割交差検証（5-Fold CV）によって比較します。
# --- 比較対象モデル ---
# 1. DynNB_Personalized  : 逐次ベイズ更新を行う、一貫性のある動的モデル
# 2. DynNB_Personalized2 : Day0のみRFのみで予測、その後更新型の動的モデル
# 3. NB_daywise          : 各日を独立に扱う、シンプルなナイーブベイズのベースライン
# 4. RF_daywise          : 各日を独立に扱う、シンプルなランダムフォレストのベースライン

# ==============================================================================
# セクション1：環境設定
# ==============================================================================
# 既存の変数を全て削除し、クリーンな状態から開始

In [ ]:
rm(list = ls())

# --- 必要なパッケージの読み込み ---
# 初回実行時は、以下のコメントを外してパッケージをインストールしてください
# install.packages(c("dplyr", "tidyr", "purrr", "e1071", "pROC", "caret", "ggplot2", "tibble", "randomForest", "rlang"))

In [ ]:
suppressPackageStartupMessages({
  library(dplyr)
  library(tidyr)
  library(purrr)
  library(e1071)
  library(pROC)
  library(caret)
  library(ggplot2)
  library(tibble)
  library(randomForest)
  library(rlang)      # `rlang::` を明示的に使うため
})

# ==============================================================================
# セクション2：データ準備
# ==============================================================================
# 生データを読み込み、モデルが学習可能な形に整形します。
# この前処理が、分析の論理的な一貫性を保証する上で最も重要です。

# --- 2.1. データの読み込み ---

In [ ]:
NB_bleeding <- read.csv("NB_bleeding_demo.csv") %>%
  arrange(id, day)

# --- 2.2. ベースデータセットの作成 ---
# 全ての計算済み列（daily_event, status_next, y_prev）を含むデータセット。
# この段階では、まだNAを含む行は削除しません。

In [ ]:
NB_bleeding_base <- NB_bleeding %>%
  filter(day <= 6) %>%
  group_by(id) %>%
  mutate(
    # 「status=1の患者の最終観察日にイベントが発生する」というルールに基づき、
    # 「その日にイベントが起きたか」を示す daily_event (0/1) を作成します。
    daily_event = if_else(status == 1 & day == max(day), 1, 0),

    # 翌日のイベント有無を示す目的変数 status_next を作成します。
    # Day t の行には、Day t+1 の daily_event の値が入ります。
    status_next = lead(daily_event),

    # 前日のイベント有無を示す説明変数 y_prev を作成します。
    y_prev = lag(daily_event)
  ) %>%
  ungroup()

# ==============================================================================
# セクション3：グローバル変数とパラメータの設定
# ==============================================================================

# --- 3.1. 説明変数リストの定義 ---

In [ ]:
static_vars  <- names(NB_bleeding)[which(names(NB_bleeding) == "age"):
                                     which(names(NB_bleeding) == "main_disease")]
dynamic_vars <- names(NB_bleeding)[which(names(NB_bleeding) == "wbc"):
                                     which(names(NB_bleeding) == "gtp")]
all_vars <- c(static_vars, dynamic_vars)
small        <- 1e-6

categorical_vars_in_static <- c("sex", "main_disease")
NB_bleeding_base[categorical_vars_in_static] <- lapply(NB_bleeding_base[categorical_vars_in_static], factor)


# ==============================================================================
# セクション4：ヘルパー関数群
# ==============================================================================
# 分析の本体で繰り返し使われる処理を、関数としてまとめて定義します。

# 条件付き統計量（平均・SD）を計算

In [ ]:
get_conditional_stats <- function(df, vars){
  `%||%` <- rlang::`%||%`
  map(vars, function(v) {
    g <- df %>% filter(!is.na(.data[[v]]), !is.na(status_next)) %>%
      group_by(status_next) %>%
      summarise(mu = mean(.data[[v]]), sd = sd(.data[[v]]), .groups = "drop")
    list(
      mean1 = g$mu[g$status_next==1][1] %||% NA_real_,
      sd1   = g$sd[g$status_next==1][1] %||% NA_real_,
      mean0 = g$mu[g$status_next==0][1] %||% NA_real_,
      sd0   = g$sd[g$status_next==0][1] %||% NA_real_
    )
  }) %>% setNames(vars)
}

# 1日分のデータから対数尤度比 (delta) を計算

In [ ]:
compute_log_likelihood_ratio <- function(row, stats){
  reduce(names(stats), .init = 0, function(acc, v) {
    s <- stats[[v]]; x <- row[[v]]
    if(!is.na(x) && !is.na(s$mean1) && !is.na(s$mean0)){
      sd1 <- max(s$sd1, small, na.rm = TRUE); sd0 <- max(s$sd0, small, na.rm = TRUE)
      acc + dnorm(x, s$mean1, sd1, log=TRUE) - dnorm(x, s$mean0, sd0, log=TRUE)
    } else { acc }
  })
}

# 安全なAUROC計算

In [ ]:
safe_auc <- function(truth, prob){
  truth <- if(is.factor(truth)) as.numeric(as.character(truth)) else truth
  if(length(unique(truth)) < 2) return(NA_real_)
  tryCatch({
    as.numeric(pROC::auc(truth, prob, quiet = TRUE, levels = c(0,1), direction = "<"))
  }, error = function(e) NA_real_)
}

# Day-wise Naive Bayes, Day-wise Random forestモデルの学習と評価

In [ ]:
train_nb_daywise <- function(train_df, test_df, vars){
  train_df <- train_df %>% select(all_of(vars), status_next) %>% na.omit()
  test_df  <- test_df  %>% select(all_of(vars), status_next) %>% na.omit()
  if(nrow(train_df)<10 || length(unique(train_df$status_next))<2 || nrow(test_df)<2) return(NA_real_)
  mdl <- naiveBayes(status_next ~ ., data = train_df %>% mutate(status_next=factor(status_next)))
  preds <- predict(mdl, test_df, type="raw")[,"1"]
  safe_auc(test_df$status_next, preds)
}

train_rf_daywise <- function(train_df, test_df, vars){
  train_df <- train_df %>% select(all_of(vars), status_next) %>% na.omit()
  test_df  <- test_df  %>% select(all_of(vars), status_next) %>% na.omit()
  if(nrow(train_df) < 10 || length(unique(train_df$status_next)) < 2 || nrow(test_df) < 2) return(NA_real_)
  mdl <- randomForest(factor(status_next) ~ ., data = train_df, ntree = 100)
  preds <- predict(mdl, newdata = test_df, type = "prob")[, "1"]
  safe_auc(test_df$status_next, preds)
}

# DynNB_Personalized モデルの予測適用関数

In [ ]:
apply_full_personalized_filter <- function(data, rf_initial_model, rf_transition_model, cond_stats, static_vars, all_vars) {
  data %>% arrange(id, day) %>%
    group_by(id) %>%
    mutate(
      score_dyn_personalized = {
        n_days <- n()
        scores <- numeric(n_days)
        day0_data_for_pred <- cur_data()[1, all_vars, drop = FALSE]
        patient_static_data <- cur_data()[1, static_vars, drop = FALSE]
        prob_init <- predict(rf_initial_model, newdata = day0_data_for_pred, type = "prob")[, "1"]
        logit_prev <- log(prob_init / (1 - prob_init))
        for (i in seq_len(n_days)) {
          y_prev_factor <- factor(round(plogis(logit_prev)), levels = c("0", "1"))
          rf_trans_input <- cbind(y_prev = y_prev_factor, patient_static_data)
          prior_t <- predict(rf_transition_model, newdata = rf_trans_input, type = "prob")[, "1"]
          logit_trans <- log(prior_t / (1 - prior_t))
          delta <- compute_log_likelihood_ratio(cur_data()[i, ], cond_stats)
          logit_now <- logit_trans + delta
          scores[i] <- logit_now
          logit_prev <- logit_now
        }
        scores
      },
      prob_dyn_personalized = plogis(score_dyn_personalized)
    ) %>%
    ungroup()
}

# DynNB_Personalized2 モデルの予測適用関数

In [ ]:
apply_dynnb_personalized2 <- function(data, rf_initial_model, rf_transition_model, cond_stats, static_vars, all_vars) {
  data %>% arrange(id, day) %>%
    group_by(id) %>%
    mutate(
      score_dyn_p2 = {
        n_days <- n()
        scores <- numeric(n_days)
        day0_data_for_pred <- cur_data()[1, all_vars, drop = FALSE]
        patient_static_data <- cur_data()[1, static_vars, drop = FALSE]
        prob_init <- predict(rf_initial_model, newdata = day0_data_for_pred, type = "prob")[, "1"]
        logit_prev <- log(prob_init / (1 - prob_init))
        for (i in seq_len(n_days)) {
          if (i == 1) {
            logit_now <- logit_prev
          } else {
            y_prev_factor <- factor(round(plogis(logit_prev)), levels = c("0", "1"))
            rf_trans_input <- cbind(y_prev = y_prev_factor, patient_static_data)
            prior_t <- predict(rf_transition_model, newdata = rf_trans_input, type = "prob")[, "1"]
            logit_trans <- log(prior_t / (1 - prior_t))
            delta <- compute_log_likelihood_ratio(cur_data()[i, ], cond_stats)
            logit_now <- logit_trans + delta
          }
          scores[i] <- logit_now
          logit_prev <- logit_now
        }
        scores
      },
      prob_dyn_p2 = plogis(score_dyn_p2)
    ) %>%
    ungroup()
}

# ==============================================================================
# セクション5：5分割交差検証によるモデル評価
# ==============================================================================

In [ ]:
set.seed(12345) # 再現性のために乱数シードを固定
k <- 5
folds <- createFolds(unique(NB_bleeding_base$id), k = k)

# 結果を格納するリストを準備
res_dyn_p <- list(); res_dyn_p2 <- list(); res_nb <- list(); res_rf_dw <- list()

message(sprintf("Running Final Corrected 5-Fold Cross-Validation with 4 models...", k))

for(f in seq_along(folds)){
  message(sprintf("Processing Fold %d/%d...", f, k))

  test_ids  <- unique(NB_bleeding_base$id)[folds[[f]]]
  train_ids <- setdiff(unique(NB_bleeding_base$id), test_ids)

  # --- 5.1. 用途別のデータ準備 ---
  # ベースデータ（NAあり）と、クリーンなデータ（NAなし）を準備
  train_data_base <- NB_bleeding_base %>% filter(id %in% train_ids)
  test_data_base  <- NB_bleeding_base %>% filter(id %in% test_ids)
  train_data <- train_data_base %>% filter(!is.na(status_next))
  test_data  <- test_data_base  %>% filter(!is.na(status_next))

  # --- 5.2. 動的モデル用のコンポーネント学習 ---
  rf_initial <- NULL; rf_transition <- NULL; st <- NULL

  train_initial_data <- train_data %>% filter(day == 0) %>% select(all_of(all_vars), status_next)
  train_transition_data <- train_data_base %>% filter(!is.na(y_prev)) %>% select(all_of(static_vars), y_prev, daily_event)

  # 学習データに陽性・陰性例が両方あるかチェックする「ガード節」
  if (length(unique(train_initial_data$status_next)) < 2 || length(unique(train_transition_data$daily_event)) < 2) {
    message(sprintf("Skipping dynamic models in Fold %d: Not enough classes for training.", f))
  } else {
    # 条件を満たす場合のみ、コンポーネントを学習
    rf_initial <- randomForest(factor(status_next) ~ ., data = train_initial_data, ntree = 100)
    train_transition_data$y_prev <- factor(train_transition_data$y_prev, levels = c("0", "1"))
    rf_transition <- randomForest(factor(daily_event) ~ ., data = train_transition_data, ntree = 100)
    st <- get_conditional_stats(train_data, dynamic_vars)
  }

  # --- 5.3. 各モデルの予測と評価 ---

  # 2a & 2b. 動的モデル群の評価
  if (!is.null(rf_initial)) {
    # 2a. DynNB_Personalized
    pred_dyn_p <- apply_full_personalized_filter(test_data, rf_initial, rf_transition, st, static_vars, all_vars)
    auc_dyn_p <- pred_dyn_p %>% group_by(day) %>%
      summarise(AUROC = safe_auc(status_next, prob_dyn_personalized), .groups="drop") %>%
      mutate(fold = f, model = "DynNB_Personalized")
    res_dyn_p[[f]] <- auc_dyn_p

    # 2b. DynNB_Personalized2
    pred_dyn_p2 <- apply_dynnb_personalized2(test_data, rf_initial, rf_transition, st, static_vars, all_vars)
    auc_dyn_p2 <- pred_dyn_p2 %>% group_by(day) %>%
      summarise(AUROC = safe_auc(status_next, prob_dyn_p2), .groups="drop") %>%
      mutate(fold = f, model = "DynNB_Personalized2")
    res_dyn_p2[[f]] <- auc_dyn_p2
  }

  # 2c. NB_daywise
  auc_nb <- map_dfr(unique(train_data$day), function(d) {
    train_d <- train_data %>% filter(day == d); test_d  <- test_data  %>% filter(day == d)
    tibble(day = d, AUROC = train_nb_daywise(train_d, test_d, all_vars))
  }) %>% mutate(fold = f, model = "NB_daywise")
  res_nb[[f]] <- auc_nb

  # 2d. RF_daywise
  auc_rf_dw <- map_dfr(unique(train_data$day), function(d) {
    train_d <- train_data %>% filter(day == d); test_d  <- test_data  %>% filter(day == d)
    tibble(day = d, AUROC = train_rf_daywise(train_d, test_d, all_vars))
  }) %>% mutate(fold = f, model = "RF_daywise")
  res_rf_dw[[f]] <- auc_rf_dw
}

message("Cross-Validation finished.")

# ==============================================================================
# セクション6：結果の集計と可視化
# ==============================================================================

# --- 6.1. 結果の集計 ---
# 5回のCVの結果を全て結合し、モデルごと・日ごとに平均AUROCと標準偏差を計算

In [ ]:
cv_results <- bind_rows(res_dyn_p, res_dyn_p2, res_nb, res_rf_dw)

agg_results <- cv_results %>%
  group_by(model, day) %>%
  summarise(
    mean_AUROC = mean(AUROC, na.rm = TRUE),
    sd_AUROC   = sd(AUROC, na.rm = TRUE),
    .groups = "drop"
  )

print("Aggregated AUROC Results:")
print(agg_results)

# --- 6.2. グラフの描画 ---
# 集計結果をプロットし、4モデルの性能を視覚的に比較

In [ ]:
ggplot(agg_results, aes(x = day, y = mean_AUROC, color = model, group = model)) +
  geom_line(linewidth = 0.8) +
  geom_point(size = 2.5) +
  geom_errorbar(
    aes(ymin = mean_AUROC - sd_AUROC, ymax = mean_AUROC + sd_AUROC),
    width = 0.2, alpha = 0.7
  ) +
  scale_y_continuous(limits = c(0.4, 1.0), breaks = seq(0.4, 1, 0.1)) +
  scale_x_continuous(breaks = 0:max(agg_results$day)) +
  labs(
    title = "5-Fold CV: Model Comparison",
    subtitle = "AUROC by Day",
    x = "Day",
    y = "Mean AUROC (±1 SD)",
    color = "Model"
  ) +
  theme_minimal(base_size = 14) +
  theme(
    plot.title = element_text(face = "bold"),
    legend.position = "bottom"
  )